<a href="https://colab.research.google.com/github/shaghayegh5ghasemi/CIFAR10_train_test/blob/main/CNN_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchsummary

**Import requirements**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchsummary import summary

**Load data and define hyperparameters**

In [2]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load our dataset
train_data = datasets.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_data, batch_size=5, shuffle=True)
test_data = datasets.CIFAR10(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=True)

# hyperparameters
learning_rate = 0.0001
batch_size = 64
num_epochs = 50

Files already downloaded and verified
Files already downloaded and verified


**Implement CNN model**

In [3]:
# our CNN model with four Layers and Relu as an activation function
class CNN(nn.Module):
  def __init__(self, channels = 3, num_classes = 10):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
    )
    self.flat = nn.Flatten()
    self.linear = nn.Linear(128*2*2, num_classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.flat(x)
    x = self.linear(x)
    x = self.softmax(x)
    return x

In [4]:
cnn_model = CNN().to(device)
summary(cnn_model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
         MaxPool2d-3           [-1, 16, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]           4,640
              ReLU-5           [-1, 32, 16, 16]               0
         MaxPool2d-6             [-1, 32, 8, 8]               0
            Conv2d-7             [-1, 64, 8, 8]          18,496
              ReLU-8             [-1, 64, 8, 8]               0
         MaxPool2d-9             [-1, 64, 4, 4]               0
           Conv2d-10            [-1, 128, 4, 4]          73,856
             ReLU-11            [-1, 128, 4, 4]               0
        MaxPool2d-12            [-1, 128, 2, 2]               0
          Flatten-13                  [-1, 512]               0
           Linear-14                   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


**Train data**

In [5]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

# train data
def train(model, train_loader, num_epochs):
  model.train()
  for epoch in range(num_epochs):
    print(f" Epoch: {epoch+1}")
    for batch_index, (data, targets) in enumerate(train_loader):
      data = data.to(device=device)
      targets = targets.to(device=device)
      output = model(data)
      loss = criterion(output, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"loss after this epoch: {loss.item()}")
    print("---------------------------------------------")

# check the acuuracy of the model
def checkaccuracy(model, loader):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on testing data")
  corrects = 0
  samples = 0
  model.eval()
  with torch.no_grad():
    for x, label in loader:
      x = x.to(device)
      label = label.to(device)
      outputs = model(x)
      _, predictions = outputs.max(1)
      corrects += (predictions == label).sum()
      samples += predictions.size(0)

  print(f'Got {corrects}/{samples} with the accuracy of {float(corrects)/float(samples)*100:.2f}')
  model.train()

In [6]:
train(cnn_model, train_loader, num_epochs)
checkaccuracy(cnn_model, test_loader)

 Epoch: 1
loss after this epoch: 2.220402240753174
---------------------------------------------
 Epoch: 2
loss after this epoch: 1.7277275323867798
---------------------------------------------
 Epoch: 3
loss after this epoch: 1.890403389930725
---------------------------------------------
 Epoch: 4
loss after this epoch: 2.291689157485962
---------------------------------------------
 Epoch: 5
loss after this epoch: 2.106693744659424
---------------------------------------------
 Epoch: 6
loss after this epoch: 1.4849392175674438
---------------------------------------------
 Epoch: 7
loss after this epoch: 1.9287641048431396
---------------------------------------------
 Epoch: 8
loss after this epoch: 1.863483190536499
---------------------------------------------
 Epoch: 9
loss after this epoch: 1.777593970298767
---------------------------------------------
 Epoch: 10
loss after this epoch: 1.6861412525177002
---------------------------------------------
 Epoch: 11
loss after thi